# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [1]:
from chrisbase.util import to_dataframe
from chrislab.common.util import GpuProjectEnv
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

with GpuProjectEnv(project_name="DeepKorean", working_gpus="0") as env:
    args = ClassificationDeployArguments(
        working_config_file=env.running_file.with_suffix('.json').name,
        pretrained_model_path="model/pretrained/KcBERT-Base",
        downstream_model_path="model/finetuned/nsmc (dl012) [03.20 21:34:29]",
        downstream_model_file=None,
        max_seq_length=128,
    )
    config = args.save_working_config()
    assert config.exists(), f"No config file: {config}"
to_dataframe(env)

downstream_model_file: epoch=2-val_loss=0.287-val_acc=0.897.ckpt


,key,value
0,hostname,dl012
1,hostaddr,129.254.182.78
2,python_path,/data/dlt/mambaforge/envs/DeepKorean-23.03/bin/python3.10
3,project_name,DeepKorean
4,project_path,/data/dlt/proj/DeepKorean-23.03
5,working_path,/data/dlt/proj/DeepKorean-23.03
6,running_file,tests/1-doc_cls-infer.ipynb
7,working_gpus,0
8,number_of_gpus,1


# 웹서비스 만들기 준비

`ngrok`은 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다.
1. Signup: [https://dashboard.ngrok.com/signup](https://dashboard.ngrok.com/signup)
2. Login : [https://dashboard.ngrok.com/login](https://dashboard.ngrok.com/login)
3. Check : [https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)
4. Config: ngrok config add-authtoken {authtoken}

In [2]:
from flask_ngrok import _download_ngrok

_download_ngrok(env.working_path / "ngrok")

In [3]:
!chmod 777 "{env.working_path}/ngrok/ngrok"
!ls -al "{env.working_path}/ngrok/ngrok"

-rwxrwxrwx 1 dlt etri 30137501 Mar 21 00:48 /data/dlt/proj/DeepKorean-23.03/ngrok/ngrok


In [4]:
your_authtoken = "2NHZJsBLbOgcBLEmZTmuvJaOJM2_2VHnFpENJjTg5dwsSLjiE"  #https://dashboard.ngrok.com/get-started/your-authtoken
!"{env.working_path}/ngrok/ngrok" authtoken {your_authtoken}

Authtoken saved to configuration file: /data/dlt/.ngrok2/ngrok.yml


In [5]:
!cat "$HOME/.ngrok2/ngrok.yml"

authtoken: 2NHZJsBLbOgcBLEmZTmuvJaOJM2_2VHnFpENJjTg5dwsSLjiE
version: "2"


In [6]:
from pathlib import Path

config = Path(config)
args = ClassificationDeployArguments.from_json(config.read_text())
to_dataframe(args)

,key,value
0,working_config_file,1-doc_cls-infer.json
1,pretrained_model_path,model/pretrained/KcBERT-Base
2,downstream_model_path,model/finetuned/nsmc (dl012) [03.20 21:34:29]
3,downstream_model_file,epoch=2-val_loss=0.287-val_acc=0.897.ckpt
4,max_seq_length,128


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [7]:
import torch
from transformers import BertConfig, BertForSequenceClassification

fine_tuned_model_ckpt = torch.load(
    Path(args.downstream_model_path) / args.downstream_model_file,
    map_location=torch.device("cpu")
)
print(f"fine_tuned_model_ckpt.keys()={fine_tuned_model_ckpt.keys()}")
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_path,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

fine_tuned_model_ckpt.keys()=dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_path,
    do_lower_case=False,
)
tokenizer

BertTokenizer(name_or_path='model/pretrained/KcBERT-Base', vocab_size=30000, model_max_length=300, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [9]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app

app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app 'ratsnlp.nlpbook.classification.deploy'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://2141-129-254-182-78.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Mar/2023 00:57:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2023 00:57:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2023 00:57:24] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2023 00:57:32] "POST /api HTTP/1.1" 200 -
